In [1]:
import torch
import numpy as np
from smolagents import HfApiModel, CodeAgent


# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64
import pandas as pd
import numpy as np
from smolagents import tool

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv()

from huggingface_hub import login

login(os.environ['HF_API_KEY'])


In [2]:
suppliers_data = {
    "name": [
        "Montreal Ice Cream Co",
        "Brain Freeze Brothers",
        "Toronto Gelato Ltd",
        "Buffalo Scoops",
        "Vermont Creamery",
    ],
    "location": [
        "Montreal, QC",
        "Burlington, VT",
        "Toronto, ON",
        "Buffalo, NY",
        "Portland, ME",
    ],
    "distance_km": [120, 85, 400, 220, 280],
    "canadian": [True, False, True, False, False],
    "price_per_liter": [1.95, 1.91, 1.82, 2.43, 2.33],
    "tasting_fee": [0, 12.50, 30.14, 42.00, 0.20],
}

data_description = """Suppliers have an additional tasting fee: that is a fixed fee applied to each order to taste the ice cream."""
suppliers_df = pd.DataFrame(suppliers_data)
suppliers_df

,name,location,distance_km,canadian,price_per_liter,tasting_fee
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20


In [3]:
# Tools

def calculate_daily_supplier_price(row):
    order_volume = 30

    # Calculate raw product cost
    product_cost = row["price_per_liter"] * order_volume

    # Calculate transport cost
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    transport_cost = row["distance_km"] * cost_per_km * trucks_needed

    # Calculate tariffs for ice cream imported from Canada
    tariff = product_cost * np.pi / 50 * row["canadian"]

    # Get total cost
    total_cost = product_cost + transport_cost + tariff + row["tasting_fee"]
    return total_cost

In [4]:
@tool
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed

@tool
def calculate_tariff(base_cost: float, is_canadian: bool) -> float:
    """
    Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.
    Assumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%

    Args:
        base_cost: the base cost of goods, not including transportation cost.
        is_canadian: wether the import is from Canada.
    """
    if is_canadian:
        return base_cost * np.pi / 50
    return 0


In [5]:
calculate_transport_cost.description

'Calculate transportation cost based on distance and order size.\nRefrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.'

In [6]:
# from helper import get_hugging_face_token

model = HfApiModel(model_id="Qwen/Qwen2.5-72B-Instruct", 
                   provider="together", 
                   # Choose a specific inference provider
                   max_tokens = 4096,
                   temperature = 0.1)


# Using the code agent

In [7]:
agent = CodeAgent(model=model, tools=[calculate_transport_cost, calculate_tariff], max_steps=10, additional_authorized_imports=["pandas", "numpy"],verbosity_level=2)
agent.logger.console.width = 66

In [8]:
agent.run("Please get me the transportation cost of 50 liters at a distance of 10 km.")

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Please get me the transportation cost of 50 liters at a        │
│ distance of 10 km.                                             │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `calculate_transport_cost` tool to get the
transportation cost for 50 liters at a distance of 10 km.         
Code:                                                             
```py                                                             
transport_cost = calculate_transport_cost(distance_km=10,         
order_volume=50)                                                  
print(transport_cost)                                             
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  transport_cost = calculate_transport_cost(distance_km=10,       
  order_volume=50)                                                
  print(transport_cost)                                           
 ────────────────────────────────────────────────────────────────

Execution logs:
12.0

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: The transportation cost for 50 liters at a distance of 10
km is 12.0. I will provide this as the final answer.              
Code:                                                             
```py                                                             
final_answer(12.0)                                                
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(12.0)                                              
 ────────────────────────────────────────────────────────────────

Out - Final answer: 12.0

[Step 2: Duration 2.04 seconds| Input tokens: 4,632 | Output 
tokens: 116]

12.0

In [9]:
task = """ This is a dataframe of different suppliers.  Please provide a comparable dataframe of the total daily price for getting daily units delivery for each of them,  given a requirement of exactly 30 liters per day.  Please take into account transportation cost and tariffs. """
agent.logger.level = 1 # Lower verbosity level

agent.run(task, additional_args={"suppliers_data":suppliers_df, "data_description":data_description})


╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ This is a dataframe of different suppliers.  Please provide a  │
│ comparable dataframe of the total daily price for getting      │
│ daily units delivery for each of them,  given a requirement of │
│ exactly 30 liters per day.  Please take into account           │
│ transportation cost and tariffs.                               │
│ You have been provided with these additional arguments, that   │
│ you can access using the keys as variables in your python      │
│ code:                                                          │
│ {'suppliers_data':                     name        location    │
│ distance_km  canadian  \                                       │
│ 0  Montreal Ice Cream Co    Montreal, QC          120          │
│ True                                                           │
│ 1  Brain Freeze Brothers  Burlington, VT           85          │
│ False                                                          │
│ 2     Toronto Gelato Ltd     Toronto, ON          400          │
│ True                                                           │
│ 3         Buffalo Scoops     Buffalo, NY          220          │
│ False                                                          │
│ 4       Vermont Creamery    Portland, ME          280          │
│ False                                                          │
│                                                                │
│    price_per_liter  tasting_fee                                │
│ 0             1.95         0.00                                │
│ 1             1.91        12.50                                │
│ 2             1.82        30.14                                │
│ 3             2.43        42.00                                │
│ 4             2.33         0.20  , 'data_description':         │
│ 'Suppliers have an additional tasting fee: that is a fixed fee │
│ applied to each order to taste the ice cream.'}.               │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  import pandas as pd                                             
                                                                  
  # Convert the suppliers_data to a DataFrame                     
  suppliers_df = pd.DataFrame(suppliers_data)                     
                                                                  
  # Calculate the transportation cost for each supplier           
  suppliers_df['transportation_cost'] = suppliers_df.apply(       
      lambda row:                                                 
  calculate_transport_cost(distance_km=row['distance_km'],        
  order_volume=30), axis=1                                        
  )                                                               
                                                                  
  # Calculate the tariff for each supplier                        
  suppliers_df['tariff'] = suppliers_df.apply(                    
      lambda row:                                                 
  calculate_tariff(base_cost=row['price_per_liter'] * 30,         
  is_canadian=row['canadian']), axis=1                            
  )                                                               
                                                                  
  # Calculate the total daily price for each supplier             
  suppliers_df['total_daily_price'] = (                           
      suppliers_df['price_per_liter'] * 30 +  # Cost of 30        
  liters of ice cream                                             
      suppliers_df['tasting_fee'] +  # Tasting fee                
      suppliers_df['transportation_cost'] +  # Transportation     
  cost                                                            
      suppliers_df['tariff']  # Tariff                            
  )                                                               
                                                                  
  # Select only the relevant columns                              
  result_df = suppliers_df[['name', 'total_daily_price']]         
                                                                  
  print(result_df)                                                
 ────────────────────────────────────────────────────────────────

Execution logs:
                    name  total_daily_price
0  Montreal Ice Cream Co         206.175663
1  Brain Freeze Brothers         171.800000
2     Toronto Gelato Ltd         568.170619
3         Buffalo Scoops         378.900000
4       Vermont Creamery         406.100000

Out: None

[Step 1: Duration 8.76 seconds| Input tokens: 2,507 | Output 
tokens: 338]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(result_df)                                         
 ────────────────────────────────────────────────────────────────

Out - Final answer:                     name  total_daily_price
0  Montreal Ice Cream Co         206.175663
1  Brain Freeze Brothers         171.800000
2     Toronto Gelato Ltd         568.170619
3         Buffalo Scoops         378.900000
4       Vermont Creamery         406.100000

[Step 2: Duration 2.32 seconds| Input tokens: 5,769 | Output 
tokens: 386]

,name,total_daily_price
0,Montreal Ice Cream Co,206.175663
1,Brain Freeze Brothers,171.800000
2,Toronto Gelato Ltd,568.170619
3,Buffalo Scoops,378.900000
4,Vermont Creamery,406.100000
